In [ ]:
import pandas as pd
import re
import numpy as np
from pathlib import Path

# --- Helpers / conversions ---
KW_TO_HP = 1.34102209          # 1 kW = 1.34102209 hp
PS_TO_HP = 0.98632007          # 1 PS (metric hp) = 0.98632007 hp
KGM_TO_NM = 9.80665            # 1 kg·m (kgm) ≈ 9.80665 N·m

def _safe_float(s):
    try:
        return float(s)
    except Exception:
        return None

# --- CLEANING FUNCTIONS ---

def clean_price(x):
    """Return numeric price (float). Accept ranges; will average. Capture decimals."""
    try:
        if pd.isna(x):
            return None
        s = str(x)
        # remove currency symbols, spaces, commas
        s = re.sub(r'[\$\€\£\₹\s,]', '', s, flags=re.UNICODE)
        # unify dashes
        s = re.sub(r'[–—−]', '-', s)
        # find floats (with decimals)
        nums = re.findall(r'\d+(?:\.\d+)?', s)
        if not nums:
            return None
        nums = [float(n) for n in nums]
        # if range (contains '-'), average, else return first
        if '-' in s and len(nums) >= 2:
            return sum(nums) / len(nums)
        return nums[0]
    except Exception:
        return None

def clean_hp(x):
    """
    Normalize HorsePower:
    - captures numbers with decimals
    - supports 'hp', 'kW', 'PS' and ranges (averaged)
    Returns hp (float) and unit label (string).
    """
    try:
        if pd.isna(x):
            return (None, None)
        s = str(x).strip()
        s_low = s.lower()
        s_low = re.sub(r'[–—−]', '-', s_low)
        # find numeric tokens (floats)
        nums = re.findall(r'\d+(?:\.\d+)?', s_low)
        if not nums:
            return (None, None)
        nums = [float(n) for n in nums]
        avg_raw = sum(nums) / len(nums)
        # unit detection & conversion
        if 'kw' in s_low:
            hp_val = avg_raw * KW_TO_HP
            return (round(hp_val, 2), 'hp (from kW)')
        if 'ps' in s_low and 'k' not in s_low:  # avoid 'kW' confusion
            hp_val = avg_raw * PS_TO_HP
            return (round(hp_val, 2), 'hp (from PS)')
        # default: treat as hp
        return (avg_raw, 'hp')
    except Exception:
        return (None, None)

def clean_torque(x):
    """
    Normalize torque to Nm when possible.
    Accepts values like "200 Nm", "20 kgm", "200-230 Nm".
    Returns (value_in_Nm, unit_label)
    """
    try:
        if pd.isna(x):
            return (None, None)
        s = str(x).strip()
        s_low = s.lower()
        s_low = re.sub(r'[–—−]', '-', s_low)
        # capture numbers with decimals
        nums = re.findall(r'\d+(?:\.\d+)?', s_low)
        if not nums:
            return (None, None)
        nums = [float(n) for n in nums]
        avg_raw = sum(nums) / len(nums)
        # unit detection
        if 'kgm' in s_low or 'kg-m' in s_low or 'kg m' in s_low:
            # convert kgm to Nm
            return (round(avg_raw * KGM_TO_NM, 2), 'Nm (from kgm)')
        if 'nm' in s_low or 'n·m' in s_low or 'n m' in s_low:
            return (avg_raw, 'Nm')
        # unknown unit - return raw numeric with None unit
        return (avg_raw, None)
    except Exception:
        return (None, None)

def clean_capacity(x):
    """
    Clean CC/Battery capacity.
    - if 'cc' present -> return cc numeric (float), unit 'cc'
    - if liters like '1.5L' or '1,5 L' -> convert to cc (liters * 1000) and unit 'cc'
    - if 'kwh' or 'kWh' or 'battery' present -> return numeric as kWh with unit 'kWh'
    - fallback: parse numeric and return as-is with unit None
    """
    try:
        if pd.isna(x):
            return (None, None)
        s = str(x).strip().lower()
        s = s.replace(',', '.')  # unify decimal separators
        s = re.sub(r'[–—−]', '-', s)
        nums = re.findall(r'\d+(?:\.\d+)?', s)
        if not nums:
            return (None, None)
        # Prefer explicit units:
        if 'cc' in s:
            return (float(nums[0]), 'cc')
        if 'l' in s and ('l' == s[-1] or re.search(r'\d+\s*l', s)):
            # assume liters -> convert to cc if the liters value is reasonable (e.g., <= 20 L)
            liters = float(nums[0])
            # if it's clearly a battery (very large) we won't convert; but assume liters for engine
            cc_val = liters * 1000.0
            return (round(cc_val, 2), 'cc')
        if 'kwh' in s or 'battery' in s or 'ah' in s:
            # treat as battery capacity, keep unit as kWh or Ah (we only extract numeric)
            return (float(nums[0]), 'kWh_or_Ah')
        # fallback: return first numeric token, unit unknown
        return (float(nums[0]), None)
    except Exception:
        return (None, None)

def clean_seats(x):
    """Return integer seats. For '5+2' or '5+1' return sum (7 or 6)."""
    try:
        if pd.isna(x):
            return None
        s = str(x)
        # capture digits
        nums = re.findall(r'\d+', s)
        if not nums:
            return None
        # if plus present, sum tokens
        if '+' in s:
            return sum(int(n) for n in nums)
        # otherwise return first integer
        return int(nums[0])
    except Exception:
        return None

def clean_numeric_str(x):
    """
    For columns like 'Total Speed', 'Performance(0 - 100 )KM/H'.
    Extract the first valid decimal number, allow multiple decimals in original by keeping first decimal point.
    """
    try:
        if pd.isna(x):
            return None
        s = str(x)
        s = s.strip()
        # remove any text except digits/dot/dash
        s = re.sub(r'[^\d\.\-]', ' ', s)
        # collapse spaces and find first float-looking token
        tokens = re.findall(r'\d+(?:\.\d+)?', s)
        if not tokens:
            return None
        # prefer the first numeric token (if it's a range like '7.2-7.4', earlier functions handle avg)
        return float(tokens[0])
    except Exception:
        return None

# --- LOAD dataset ---
in_path = Path("input/cardataset.csv")
if not in_path.exists():
    raise FileNotFoundError(f"{in_path} not found. Adjust the path or upload the file.")

df = pd.read_csv(in_path, encoding="latin1")

# --- Columns existence check ---
cols_expected = ["Cars Prices", "Torque", "HorsePower", "CC/Battery Capacity", "Seats",
                 "Total Speed", "Performance(0 - 100 )KM/H"]
missing_cols = [c for c in cols_expected if c not in df.columns]
if missing_cols:
    print("Warning - these expected columns were not found and will be skipped:", missing_cols)

# --- APPLY CLEANING (safe) ---
# Price
if "Cars Prices" in df.columns:
    df["Cars Prices_cleaned"] = df["Cars Prices"].apply(clean_price)

# Horsepower -> returns tuple (hp_value, unit_label)
if "HorsePower" in df.columns:
    hp_parsed = df["HorsePower"].apply(clean_hp)
    df["HorsePower_hp"] = hp_parsed.apply(lambda t: t[0])
    df["HorsePower_unit"] = hp_parsed.apply(lambda t: t[1])

# Torque -> returns tuple (Nm_value, unit_label)
if "Torque" in df.columns:
    torque_parsed = df["Torque"].apply(clean_torque)
    df["Torque_Nm"] = torque_parsed.apply(lambda t: t[0])
    df["Torque_unit"] = torque_parsed.apply(lambda t: t[1])

# CC/Battery -> returns (value, unit)
if "CC/Battery Capacity" in df.columns:
    cap_parsed = df["CC/Battery Capacity"].apply(clean_capacity)
    df["CC_Battery_value"] = cap_parsed.apply(lambda t: t[0])
    df["CC_Battery_unit"] = cap_parsed.apply(lambda t: t[1])

# Seats
if "Seats" in df.columns:
    df["Seats_cleaned"] = df["Seats"].apply(clean_seats)

# Numeric strings
for col in ["Total Speed", "Performance(0 - 100 )KM/H"]:
    if col in df.columns:
        df[col + "_cleaned"] = df[col].apply(clean_numeric_str)

# --- Post-checks / conversions: replace NaN-like with None for readability ---
df = df.replace({np.nan: None})

# --- Summary of changes ---
print("Preview of cleaned columns (first 10 rows):")
preview_cols = [c for c in df.columns if any(k in c for k in ["cleaned", "hp", "Torque_Nm", "CC_Battery", "Cars Prices"])]
print(df[preview_cols].head(10).to_string())

# --- Save cleaned result ---
out_dir = Path("/mnt/data")
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "cardataset_cleaned.csv"
df.to_csv(out_path, index=False)
print(f"Cleaned dataset saved to: {out_path}")


Dataset shape (rows, columns): (32, 12)

--- First 5 records ---


,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2



--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  32 non-null     object 
 1   mpg         32 non-null     float64
 2   cyl         32 non-null     int64  
 3   disp        32 non-null     float64
 4   hp          32 non-null     int64  
 5   drat        32 non-null     float64
 6   wt          32 non-null     float64
 7   qsec        32 non-null     float64
 8   vs          32 non-null     int64  
 9   am          32 non-null     int64  
 10  gear        32 non-null     int64  
 11  carb        32 non-null     int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 3.1+ KB

--- Summary Statistics ---


,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
count,32,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.0000
unique,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Mazda RX4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,20.090625,6.187500,230.721875,146.687500,3.596563,3.217250,17.848750,0.437500,0.406250,3.687500,2.8125
std,NaN,6.026948,1.785922,123.938694,68.562868,0.534679,0.978457,1.786943,0.504016,0.498991,0.737804,1.6152
min,NaN,10.400000,4.000000,71.100000,52.000000,2.760000,1.513000,14.500000,0.000000,0.000000,3.000000,1.0000
25%,NaN,15.425000,4.000000,120.825000,96.500000,3.080000,2.581250,16.892500,0.000000,0.000000,3.000000,2.0000
50%,NaN,19.200000,6.000000,196.300000,123.000000,3.695000,3.325000,17.710000,0.000000,0.000000,4.000000,2.0000
75%,NaN,22.800000,8.000000,326.000000,180.000000,3.920000,3.610000,18.900000,1.000000,1.000000,4.000000,4.0000



Number of duplicate rows: 0
Shape after removing duplicates: (32, 12)

--- Unique values in categorical columns ---


KeyError: 'Car_Name'